In [1]:
import pandas as pd
from utils import data_util as du, preproc_utils as pu, extractor_util as exu, estimator_util as esu, evaluation_util as evu
import importlib
from typing import List, Dict, Any
import wrappers as wrp
import os
import numpy as np

In [2]:
DATA_SRC_DIR = '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data'
EXT_DATA_DIR = '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/external'
INT_DATA_DIR = '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/internal'

In [3]:
importlib.reload(wrp)
importlib.reload(du)
importlib.reload(exu)
importlib.reload(esu)

<module 'utils.estimator_util' from '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/utils/estimator_util.py'>

In [5]:
# Usage
config = pu.PreprocessConfig(size=(512, 512), normalize=False)
preprocessor = pu.ImagePreprocessor(config)

In [6]:
default_sift_config = exu.SIFTConfig()
extractor_config = exu.FeatureExtractorConfig(default_sift_config)
extractor = exu.FeatureExtractor(config=extractor_config)

In [7]:
dataset = wrp.load_dataset(EXT_DATA_DIR)

In [8]:
dataset = wrp.preprocess_data(dataset, preprocessor, extractor)

In [9]:
importlib.reload(wrp)

wrp.sample_pairs_for_run(dataset, 40)

[-] Processing scene "brandenburg_gate": found 6767 pairs (will keep 40)
[-] Processing scene "trevi_fountain": found 11166 pairs (will keep 40)
[-] Processing scene "lincoln_memorial_statue": found 6308 pairs (will keep 40)
[-] Processing scene "pantheon_exterior": found 9888 pairs (will keep 40)
[-] Processing scene "buckingham_palace": found 6646 pairs (will keep 40)
[-] Processing scene "taj_mahal": found 6730 pairs (will keep 40)
[-] Processing scene "temple_nara_japan": found 8115 pairs (will keep 40)
[-] Processing scene "sagrada_familia": found 2767 pairs (will keep 40)
[-] Processing scene "notre_dame_front_facade": found 8448 pairs (will keep 40)
[-] Processing scene "colosseum_exterior": found 7314 pairs (will keep 40)
[-] Processing scene "sacre_coeur": found 4233 pairs (will keep 40)
[-] Processing scene "british_museum": found 889 pairs (will keep 40)


In [10]:
dataset = wrp.extract_features(dataset, extractor)

In [11]:
bf_config = exu.BFMatcherConfig(knn=True)
matcher_config = exu.MatcherConfig(bf_config)

In [12]:
matcher = exu.FeatureMatcher(matcher_config)

In [13]:
dataset.scenes_data['brandenburg_gate'].image_data['27175827_2221395482'].for_exp

0

In [14]:
importlib.reload(wrp)
importlib.reload(exu)

matcher = exu.FeatureMatcher(matcher_config)
wrp.match_features(dataset, matcher, covisibility_threshold=0.1)


Matching features for scene: brandenburg_gate
In matcher there are 40 valid pairs to estimate for
60770994_853214983 08814095_3507579332
82795354_2835662202 76144572_5654283700
76144572_5654283700 05712502_4051278060
76144572_5654283700 66720928_3418295149
70714847_7062225869 18350037_3577554432
62661350_182339698 38600512_2168650655
66720928_3418295149 48881630_6195994710
67814984_5083477916 54870639_2216920778
45962310_5550442083 05712502_4051278060
53543924_286554986 10605892_4176363979
59826471_8014732885 37403694_5070563509
62661350_182339698 33115046_5083480634
95215027_2335003350 05714706_2362913557
95048461_7775852580 77495133_2091260914
93505867_8891475944 77037144_51142129
76602992_5007474555 09840944_156579676
67814984_5083477916 20516163_2396433020
33115046_5083480634 25378482_2036660966
37209818_7325320236 28091537_2961893803
45962310_5550442083 14932525_2597278914
94235774_3066657579 37403694_5070563509
94235774_3066657579 67266149_8114963267
88312545_2361506957 70714847_

In [15]:
list1 = [(1, 2), (3, 4), (5, 6), (7, 8), (9, 10), (11, 12), (13, 14), (15, 16)]
list2 = [(17, 18), (19, 20), (21, 22), (23, 24), (25, 26), (27, 28), (29, 30), (31, 32)]

In [16]:
dataset.scenes_data['lincoln_memorial_statue'].covisibility[dataset.scenes_data['lincoln_memorial_statue'].covisibility.for_exp == 1].head(2)

,covisibility,fundamental_matrix,im1,im2,for_exp,keypoints1,keypoints2
pair,,,,,,,
33905156_10379878755-03761674_4598051029,0.841,-5.07282202e-02 -1.57534652e+00 2.20767235e+03...,33905156_10379878755,03761674_4598051029,1.0,"(-0.233216, -0.317427);(-0.233216, -0.317427);...","(-0.175244, -0.297269);(-0.175244, -0.297269);..."
28493739_7144738253-25063495_13061263145,0.817,-1.98162304e-01 -2.88207148e+00 3.43138678e+03...,28493739_7144738253,25063495_13061263145,1.0,"(-0.169527, -0.232515);(-0.153926, -0.237544);...","(-0.078644, -0.189739);(-0.071484, -0.184241);..."


In [17]:
importlib.reload(wrp)
 

<module 'wrappers' from '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/wrappers.py'>

In [18]:
importlib.reload(esu)

# Recreate the estimator configuration and estimator using the reloaded module
es_alg_conf = esu.RANSACConfig()
es_conf = esu.EstimatorConfig('RANSAC', es_alg_conf)
fe = esu.FundamentalMatrixEstimator(list1, list2, es_conf)


In [19]:
sub = wrp.estimate_fundamental_matrix(dataset=dataset, estimator=fe)

Estimating fundamental matrix for pairs in scene: brandenburg_gate
Estimating fundamental matrix for pairs in scene: trevi_fountain
Estimating fundamental matrix for pairs in scene: lincoln_memorial_statue
Estimating fundamental matrix for pairs in scene: pantheon_exterior
Estimating fundamental matrix for pairs in scene: buckingham_palace
Estimating fundamental matrix for pairs in scene: taj_mahal
Estimating fundamental matrix for pairs in scene: temple_nara_japan
Estimating fundamental matrix for pairs in scene: sagrada_familia
Estimating fundamental matrix for pairs in scene: notre_dame_front_facade
Estimating fundamental matrix for pairs in scene: colosseum_exterior
Estimating fundamental matrix for pairs in scene: sacre_coeur
Estimating fundamental matrix for pairs in scene: british_museum


In [20]:
sub

,sample_id,fundamental_matrix,mask,inliers1,inliers2
0,brandenburg_gate;60770994_853214983-08814095_3...,"[8.994987408362828, 15.077973959771438, 6.4675...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.548892, -0.039872], [-0.539844, -0.200452...","[[-0.4328, -0.204159], [-0.45194, -0.198876], ..."
1,brandenburg_gate;82795354_2835662202-76144572_...,"[22.148424065408037, 32.39294580447763, 7.7661...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.432461, -0.135804], [-0.432461, -0.135804...","[[-0.272611, -0.092036], [-0.272611, -0.092036..."
2,brandenburg_gate;76144572_5654283700-05712502_...,"[-42.102145292684, -70.04660290918271, -17.550...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.301226, 0.026772], [-0.297744, 0.036877],...","[[-0.23234, -0.045125], [-0.231155, -0.038193]..."
3,brandenburg_gate;76144572_5654283700-66720928_...,"[68.83053117537582, -162.21607539358112, -17.2...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.297426, 0.026874], [-0.287713, 0.023979],...","[[-0.035937, 0.093438], [-0.126965, -0.001623]..."
4,brandenburg_gate;70714847_7062225869-18350037_...,"[1.3530850771689473, 167.22128044823663, 57.52...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.302365, -0.117508], [-0.259624, -0.024402...","[[-0.117283, -0.073053], [-0.150864, -0.296008..."
...,...,...,...,...,...
475,british_museum;80976991_8342057400-66552192_43...,"[-0.40812353825906916, -6.36002944657814, 0.02...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.359505, 0.059381], [-0.351762, 0.124706],...","[[-0.580548, -0.338968], [-0.454656, 0.139079]..."
476,british_museum;88500284_27304784-03576546_3676...,"[5.114703918903418, -87.44746543569525, -3.571...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.239194, -0.099984], [-0.239194, -0.099984...","[[-0.377352, 0.032562], [-0.377352, 0.032562],..."
477,british_museum;88500284_27304784-62553151_8395...,"[-4.331976491301295, -142.82597351656312, -5.9...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.236828, -0.00198], [-0.23412, -0.014208],...","[[-0.325298, -0.1416], [-0.323302, -0.151322],..."
478,british_museum;77723525_5227836172-73748105_45...,"[10.986869874048828, 25.151029681826483, 12.51...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.645801, -0.191352], [-0.644345, -0.134007...","[[-0.052652, 0.009423], [-0.120174, 0.059481],..."


In [21]:
sub[sub.fundamental_matrix.isna()]

,sample_id,fundamental_matrix,mask,inliers1,inliers2
115,lincoln_memorial_statue;44904301_532765055-201...,None,"[[0], [0], [0], [0], [0], [0], [0], [0], [0], ...",[],[]
117,lincoln_memorial_statue;81297701_879934582-704...,None,"[[0], [0], [0], [0], [0], [0], [0], [0], [0]]",[],[]


In [22]:
scaling_data = pd.read_csv(os.path.join(EXT_DATA_DIR, 'train/scaling_factors.csv'), index_col=0).set_index('scene')

In [23]:
thresholds_q = np.linspace(1, 10, 10)
thresholds_t = np.geomspace(0.2, 5, 10)

In [24]:
importlib.reload(wrp)

evals = wrp.evaluate_results(dataset, sub, scaling=scaling_data, thresholds_q=thresholds_q, thresholds_t=thresholds_t)

Evaluating for brandenburg_gate
Evaluating for trevi_fountain
Evaluating for lincoln_memorial_statue
Evaluating for pantheon_exterior
Evaluating for buckingham_palace
Evaluating for taj_mahal
Evaluating for temple_nara_japan
Evaluating for sagrada_familia
Evaluating for notre_dame_front_facade
Evaluating for colosseum_exterior
Evaluating for sacre_coeur
Evaluating for british_museum


In [25]:
# Extract scene names from sample_id
evals['scene'] = evals['sample_id'].apply(lambda x: x.split(';')[0])

# Group by scene and calculate mean accuracy measures
scene_accuracy = evals.groupby('scene').agg({
    'maa': 'mean'
}).reset_index()

print(scene_accuracy)

                      scene     maa
0          brandenburg_gate  0.0075
1            british_museum  0.0275
2         buckingham_palace  0.0025
3        colosseum_exterior  0.0000
4   lincoln_memorial_statue  0.0500
5   notre_dame_front_facade  0.0050
6         pantheon_exterior  0.0025
7               sacre_coeur  0.0000
8           sagrada_familia  0.0150
9                 taj_mahal  0.0575
10        temple_nara_japan  0.0100
11           trevi_fountain  0.0000


In [46]:
importlib.reload(du)

test_data = du.DatasetLoader(os.path.join(EXT_DATA_DIR), train_mode=False)


In [47]:
importlib.reload(wrp)

test_data = wrp.preprocess_data(test_data, preprocessor, extractor)

In [50]:
test_data.scenes_data['st_peters_square']

SceneData(images_dir=PosixPath('/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/external/test_images/st_peters_square'), calibration=Empty DataFrame
Columns: []
Index: [], covisibility=Empty DataFrame
Columns: []
Index: [], image_data={'55625141_5722172320': ImageData(name='55625141_5722172320', path=PosixPath('/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/external/test_images/st_peters_square/55625141_5722172320.jpg'), preproc_contents=array([[[222, 182, 134],
        [222, 182, 134],
        [221, 182, 133],
        ...,
        [213, 166, 115],
        [213, 166, 115],
        [213, 166, 115]],

       [[222, 182, 134],
        [222, 182, 134],
        [222, 182, 134],
        ...,
        [213, 166, 115],
        [213, 166, 115],
        [213, 166, 115]],

       [[223, 183, 135],
        [223, 183, 135],
        [223, 183, 135],
        ...,
        [213, 166, 115],
        [213, 166, 115],
        [213, 166, 115]],

       ...,

       [[ 28,  29

In [51]:
test_data = wrp.extract_features(test_data, extractor)